In [66]:
#Import the NFL play by play data
import pandas as pd
from pandas import DataFrame

#Enter desired years of data
YEARS = [2019,2018,2017]

data = pd.DataFrame()

for i in YEARS:  
    #low_memory=False eliminates a warning
    i_data = pd.read_csv('https://github.com/nflverse/nflfastR-data/blob/master/data/' \
                         'play_by_play_' + str(i) + '.csv.gz?raw=True',
                         compression='gzip', low_memory=False)

    #sort=True eliminates a warning and alphabetically sorts columns
    data = data.append(i_data, sort=True)

#Give each row a unique index
data.reset_index(drop=True, inplace=True)

In [67]:
list(data.columns)

['aborted_play',
 'air_epa',
 'air_wpa',
 'air_yards',
 'assist_tackle',
 'assist_tackle_1_player_id',
 'assist_tackle_1_player_name',
 'assist_tackle_1_team',
 'assist_tackle_2_player_id',
 'assist_tackle_2_player_name',
 'assist_tackle_2_team',
 'assist_tackle_3_player_id',
 'assist_tackle_3_player_name',
 'assist_tackle_3_team',
 'assist_tackle_4_player_id',
 'assist_tackle_4_player_name',
 'assist_tackle_4_team',
 'away_coach',
 'away_score',
 'away_team',
 'away_timeouts_remaining',
 'away_wp',
 'away_wp_post',
 'blocked_player_id',
 'blocked_player_name',
 'comp_air_epa',
 'comp_air_wpa',
 'comp_yac_epa',
 'comp_yac_wpa',
 'complete_pass',
 'cp',
 'cpoe',
 'def_wp',
 'defensive_extra_point_attempt',
 'defensive_extra_point_conv',
 'defensive_two_point_attempt',
 'defensive_two_point_conv',
 'defteam',
 'defteam_score',
 'defteam_score_post',
 'defteam_timeouts_remaining',
 'desc',
 'div_game',
 'down',
 'drive',
 'drive_end_transition',
 'drive_end_yard_line',
 'drive_ended_with_

In [68]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143905 entries, 0 to 143904
Columns: 370 entries, aborted_play to yrdln
dtypes: float64(177), int64(38), object(155)
memory usage: 406.2+ MB


In [69]:
data.head()

,aborted_play,air_epa,air_wpa,air_yards,assist_tackle,assist_tackle_1_player_id,assist_tackle_1_player_name,assist_tackle_1_team,assist_tackle_2_player_id,assist_tackle_2_player_name,...,xyac_median_yardage,xyac_success,yac_epa,yac_wpa,yardline_100,yards_after_catch,yards_gained,ydsnet,ydstogo,yrdln
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,MIN 35
1,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,35.0,NaN,0.0,8.0,0,MIN 35
2,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,75.0,NaN,-8.0,8.0,10,ATL 25
3,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,83.0,NaN,4.0,8.0,18,ATL 17
4,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,79.0,NaN,12.0,8.0,14,ATL 21


In [87]:
third_downs_with_nan = data[data.down == 3.0]
third_downs_with_nan.head()

,aborted_play,air_epa,air_wpa,air_yards,assist_tackle,assist_tackle_1_player_id,assist_tackle_1_player_name,assist_tackle_1_team,assist_tackle_2_player_id,assist_tackle_2_player_name,...,xyac_median_yardage,xyac_success,yac_epa,yac_wpa,yardline_100,yards_after_catch,yards_gained,ydsnet,ydstogo,yrdln
4,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,79.0,NaN,12.0,8.0,14,ATL 21
13,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,77.0,NaN,0.0,12.0,3,ATL 23
23,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,70.0,NaN,0.0,34.0,5,ATL 30
26,0,1.520888,0.0,5.0,0.0,NaN,NaN,NaN,NaN,NaN,...,2.0,0.97492,0.146845,0.016882,60.0,1.0,6.0,34.0,5,ATL 40
32,0,3.544151,0.0,21.0,0.0,NaN,NaN,NaN,NaN,NaN,...,2.0,1.00000,-5.125284,-0.012735,41.0,NaN,0.0,34.0,19,MIN 41


In [90]:
third_downs = third_downs_with_nan.columns[third_downs_with_nan.isnull().mean() < 0.5]

In [91]:
third_downs.head()

AttributeError: 'Index' object has no attribute 'head'

In [ ]:
missing = pd.concat([ski_data.isnull().sum(), 100 * ski_data.isnull().mean()], axis=1)
missing.columns=['Sum', 'Mean']
missing.sort_values(by='Sum', ascending=False)

In [ ]:
list(missing_values_df.columns)

In [ ]:
missing = missing.drop(columns=(missing_values_df[['Mean' == False]]))

In [ ]:
third_downs.drop(columns = ['aborted_play'])

In [ ]:
for x in new_data:
    missing = x.isna().sum()
    if missing > len(data.index)/2:
        new_data.drop(x, inplace=True)        
        

- keep only rows that are 3rd down plays
- need results from first and second downs into a column
- every third down has it own play ID
- seperate data frame for 3rd downs
- keep only column that i see as valuable in current state or any columns that may be usful if i change them 
- a way to reduce the number of columns is look at the percentage of missing values
    - if column is missing more than 50% of values then it is hard to do anything with column
    